In [1]:
import pandas as pd
import pickle

# Save dataframe to pickled pandas object
# df.to_pickle(file_name) # where to save it usually as a .pkl

# Load dataframe from pickled pandas object
df = pd.read_pickle('/content/roitimeseriesdata.pkl')

# Rename the third column to "ROIxTimeseries"
df = df.rename(columns={df.columns[2]: "ROIxTimeseries"})

print(df)

   site patient_id                                     ROIxTimeseries label
0   NYU      50965  [[0.706183, 1.258003, 0.042666, -2.820356, -5....     1
1   NYU      50990  [[-3.279094, -6.948174, -8.92033, -8.804001, -...     1
2   NYU      50957  [[9.66222, 8.258377, 2.37984, -3.478141, -6.05...     1
3   NYU      50967  [[4.721457, 7.912924, 8.742649, 6.427285, 2.21...     1
4   NYU      50974  [[5.851009, 7.150782, 4.961509, 0.283124, -4.0...     1
5   NYU      51013  [[-1.481923, -0.191252, -1.457838, -5.373023, ...     1
6   NYU      51015  [[-1.754589, -2.926472, -4.571375, -4.713179, ...     1
7   NYU      50996  [[4.428881, 1.885072, -3.001936, -6.947103, -7...     1
8   NYU      50977  [[5.811244, 9.114678, 6.911661, 0.606224, -4.1...     1
9   NYU      50992  [[1.321344, -1.506006, -4.413543, -4.332966, -...     1
10  NYU      51007  [[5.25749, 12.46537, 14.226592, 7.527265, -4.3...     1
11  NYU      51012  [[9.326157, 7.579217, 1.68043, -2.946055, -2.4...     1
12  NYU     

In [2]:
# Assuming your DataFrame is named df
third_column = df['ROIxTimeseries']  # Selecting all rows and the third column (index 2)

print(third_column)

0     [[0.706183, 1.258003, 0.042666, -2.820356, -5....
1     [[-3.279094, -6.948174, -8.92033, -8.804001, -...
2     [[9.66222, 8.258377, 2.37984, -3.478141, -6.05...
3     [[4.721457, 7.912924, 8.742649, 6.427285, 2.21...
4     [[5.851009, 7.150782, 4.961509, 0.283124, -4.0...
5     [[-1.481923, -0.191252, -1.457838, -5.373023, ...
6     [[-1.754589, -2.926472, -4.571375, -4.713179, ...
7     [[4.428881, 1.885072, -3.001936, -6.947103, -7...
8     [[5.811244, 9.114678, 6.911661, 0.606224, -4.1...
9     [[1.321344, -1.506006, -4.413543, -4.332966, -...
10    [[5.25749, 12.46537, 14.226592, 7.527265, -4.3...
11    [[9.326157, 7.579217, 1.68043, -2.946055, -2.4...
12    [[1.884253, 4.035859, 3.173807, -0.894492, -5....
13    [[-13.943059, -17.944493, -5.253335, 13.283772...
14    [[4.396806, 1.299931, -3.178451, -5.013186, -2...
15    [[1.48535, 2.884969, 1.535843, -1.060772, -1.2...
16    [[-0.872018, -6.980013, -10.826739, -10.537724...
17    [[5.151979, 3.185684, -2.209435, -5.959943

In [3]:
# Saving the column to a CSV file
# third_column.to_csv('ROIxTimeseries.csv', index=False)

In [4]:
# Retrieve the third column for the first row as an array
roi_signal_array = df.iloc[1, 2]

roi_signal_array.shape

(116, 176)

In [5]:
roi_signal_array

array([[ -3.279094,  -6.948174,  -8.92033 , ...,   5.281155,   2.413151,
         -1.649647],
       [ -6.112521,  -8.576906,  -9.744708, ...,   3.876581,   2.191601,
          0.324752],
       [  1.562248,   2.957436,   3.395749, ...,   0.814989,   5.289425,
          5.154504],
       ...,
       [  1.120911,   8.668012,   4.149826, ...,   1.830852,   6.668108,
          7.39312 ],
       [ 31.875685,  17.376093, -17.544689, ..., -21.662167, -35.424531,
        -26.395877],
       [-18.387568,  -9.9117  ,  20.975327, ..., -38.063708, -37.438396,
        -19.627808]])

In [6]:
import numpy as np

Generating Matrices now

In [7]:
import matplotlib.pyplot as plt

In [8]:
import numpy as np
from scipy.signal import hilbert

In [9]:
def psi(x, y):
  """Calculate the phase synchronization index between two signals.

  Args:
    x: A NumPy array representing the first signal.
    y: A NumPy array representing the second signal.

  Returns:
    A NumPy array representing the PSI between the two signals.
  """

  # Calculate the analytic representation of the signals.
  x_analytic = hilbert(x)
  y_analytic = hilbert(y)

  # Calculate the instantaneous phase of the signals.
  x_phase = np.angle(x_analytic)
  y_phase = np.angle(y_analytic)

  # print("Phases:\n X:",x_phase.shape, "Y:",y_phase.shape)
  # print("DIFF in phase : ", x_phase - y_phase)
  # print("Cosine : ", np.cos(x_phase-y_phase))
  # Calculate the PSI.
  psi = abs(np.mean(np.cos(x_phase - y_phase)))

  return psi

For all patients

In [10]:
# Function to convert matrix rows [all rows] into separate arrays
def expand_matrix_row(row):
    arrays = row['ROIxTimeseries']
    patient_id = row['patient_id']
    new_rows = [{'patient_id': patient_id, 'Array': np.array(arr)} for arr in arrays]
    return pd.DataFrame(new_rows)

num_rows = len(df)

# Initialize an empty list to store the results
result_list = []

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    # Expand the matrix rows into separate arrays for the current row
    expanded_df = expand_matrix_row(row)

    # Initialize an empty matrix to store PSI values for the current row
    num_rows_expanded = len(expanded_df)
    psi_matrix_1pat = np.zeros((num_rows_expanded, num_rows_expanded))

    # Loop through every pair of rows
    for i in range(num_rows_expanded):
        for j in range(i, num_rows_expanded):
            # Ensure not to repeat pairs
            if i == j:
                # Calculate the diagonal element
                x = expanded_df.iloc[i]['Array']
                psi_value = psi(x, x)
                psi_matrix_1pat[i, j] = psi_value
            else:
                # Extract signals for the current pair of rows
                x = expanded_df.iloc[i]['Array']
                y = expanded_df.iloc[j]['Array']
                # Calculate the PSI value for the current pair of signals
                psi_value = psi(x, y)
                # Store the PSI value in the matrix
                psi_matrix_1pat[i, j] = psi_value
                psi_matrix_1pat[j, i] = psi_value  # Since PSI is symmetric, store in both positions

            psi_matrix_1pat.shape

    # Append the PSI matrix and label to the result list
    result_list.append({'psi_matrix': psi_matrix_1pat, 'label': row['label']})

# Create a DataFrame from the result list
result_df = pd.DataFrame(result_list)

In [11]:
print(result_df.shape)

(42, 2)


In [12]:
# Get the matrix from the first row (row index 0)
matrix_row_1 = result_df.loc[0, 'psi_matrix']

# Get the shape (dimensions) of the matrix
matrix_shape_row_1 = matrix_row_1.shape

print(f"The shape of the matrix in row 1 is: {matrix_shape_row_1}")

The shape of the matrix in row 1 is: (116, 116)


In [13]:
# Get the matrix from the first row (row index 0)
matrix_row_1 = result_df.loc[0, 'psi_matrix']

# Print the matrix
print(matrix_row_1)

[[1.         0.25906469 0.06693214 ... 0.13713135 0.01496673 0.05904445]
 [0.25906469 1.         0.21868974 ... 0.01127428 0.10975937 0.05835404]
 [0.06693214 0.21868974 1.         ... 0.18562702 0.03774552 0.18821166]
 ...
 [0.13713135 0.01127428 0.18562702 ... 1.         0.30204957 0.23777238]
 [0.01496673 0.10975937 0.03774552 ... 0.30204957 1.         0.21706852]
 [0.05904445 0.05835404 0.18821166 ... 0.23777238 0.21706852 1.        ]]


Padding Matrices (224 x 224)

In [14]:
import numpy as np
import pandas as pd
import os

In [15]:
def pad_matrix(row):
    matrix = row['psi_matrix']
    pad_top = (224 - matrix.shape[0]) // 2
    pad_bottom = 224 - matrix.shape[0] - pad_top
    pad_left = (224 - matrix.shape[1]) // 2
    pad_right = 224 - matrix.shape[1] - pad_left

    padded_matrix = np.pad(matrix, ((pad_top, pad_bottom), (pad_left, pad_right)), mode='constant')
    return padded_matrix

# Assuming your original DataFrame is named 'df'
new_df = result_df.copy()
new_df['psi_matrix'] = new_df.apply(lambda row: pad_matrix(row), axis=1)

In [16]:
print(new_df.shape)

(42, 2)


In [17]:
# Get the matrix from the first row (row index 0)
matrix_row_1 = new_df.loc[0, 'psi_matrix']

# Get the shape (dimensions) of the matrix
matrix_shape_row_1 = matrix_row_1.shape

print(f"The shape of the matrix in row 1 is: {matrix_shape_row_1}")

The shape of the matrix in row 1 is: (224, 224)


In [18]:
# Get the matrix from the first row (row index 0)
matrix_row_1 = new_df.loc[0, 'psi_matrix']

# Print the matrix
print(matrix_row_1)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [19]:
# Check the padded matrix for row 41
row_index = 40  # Row index starts from 0, so 40 corresponds to the 41st row
padded_matrix = new_df.iloc[row_index]['psi_matrix']
label = new_df.iloc[row_index]['label']

In [20]:
# Save the padded matrix as a CSV file
filename = f"padded_matrix_row_{row_index + 1}_label_{label}.csv"
np.savetxt(filename, padded_matrix, delimiter=',')
print(f"Saved padded matrix for row {row_index + 1} with label {label} to {filename}")

Saved padded matrix for row 41 with label 0 to padded_matrix_row_41_label_0.csv


Dataset preparation

In [21]:
# Drop the first 30 rows from the original DataFrame
new_df = new_df.drop(df.index[:12])

print(new_df.shape)

(30, 2)


In [22]:
X = np.array(new_df['psi_matrix'])

In [23]:
X

array([array([[0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.],
              ...,
              [0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.]]),
       array([[0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.],
              ...,
              [0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.]]),
       array([[0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.],
              ...,
              [0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.]]),
       array([[0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0

In [24]:
y = new_df['label']

In [25]:
y

12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    1
21    1
22    1
23    1
24    1
25    1
26    1
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
Name: label, dtype: int64

Deep Learning Model

In [26]:
import numpy as np

In [27]:
import tensorflow as tf

In [28]:
from tensorflow.keras.optimizers import Adam

In [29]:
from keras.applications.vgg16 import preprocess_input

In [30]:
# VGG model expects a 4D tensor as input, with the shape (batch_size, height, width, channels).
# Then preprocess the input data using preprocess_input from the keras.applications.vgg16 module, which applies the necessary preprocessing steps required by the VGG model.
X = np.stack(new_df['psi_matrix'].values).reshape(-1, 224, 224)

In [31]:
X.shape

(30, 224, 224)

In [32]:
# Expand the last dimension to include the channel dimension
X = np.expand_dims(X, axis=-1)

In [33]:
X.shape

(30, 224, 224, 1)

In [34]:
# Repeat the data for each channel (RGB)
X = np.repeat(X, 3, axis=-1)
X.shape

# The line X = np.repeat(X, 3, axis=-1) repeats the data in the last dimension (the channel dimension) three times. This is necessary because the VGG pretrained model expects input data with 3 channels (RGB).

# Since your original data is a 2D matrix, it only has one channel. By repeating the data three times, you are creating a new 3D tensor with three channels, each containing the same data.

# Here is a breakdown of what the code does:

# np.repeat(X, 3, axis=-1):
# X: The input data, a 3D tensor with dimensions (batch_size, height, width, 1).
# 3: The number of times to repeat the data along the last dimension.
# axis=-1: The axis along which to repeat the data. In this case, the last dimension (channel dimension).
# The output of the code is a 3D tensor with dimensions (batch_size, height, width, 3), where each channel contains the same data as the original 2D matrix.

# This allows you to use the VGG pretrained model with your data, as it expects input data with 3 channels.

(30, 224, 224, 3)

In [35]:
# Skipping this step (as PSI matrix contains original values)

# preprocessing according to vgg (requires 3 channels)
# X = preprocess_input(X)  # Preprocess the input data for VGG

In [36]:
y = new_df['label'].values

In [37]:
X.shape

(30, 224, 224, 3)

In [38]:
y.shape

(30,)

In [39]:
X

array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        ...,

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]],


       [[[0., 0., 0.],
         [0., 0., 0.],
         [0., 

In [40]:
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Flatten
from keras.models import Model

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add classification layers
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

58889256/58889256 [==============================] - 0s 0us/step


not performing fine tuning during training

In [43]:
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [44]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=8)

Epoch 1/20
3/3 [==============================] - 9s 1s/step - loss: 7.6729 - accuracy: 0.2917 - val_loss: 3.2477 - val_accuracy: 0.3333
Epoch 2/20
3/3 [==============================] - 0s 75ms/step - loss: 3.7063 - accuracy: 0.5417 - val_loss: 3.2877 - val_accuracy: 0.3333
Epoch 3/20
3/3 [==============================] - 0s 76ms/step - loss: 1.1832 - accuracy: 0.7083 - val_loss: 1.8539 - val_accuracy: 0.6667
Epoch 4/20
3/3 [==============================] - 0s 74ms/step - loss: 3.1473 - accuracy: 0.5417 - val_loss: 0.9169 - val_accuracy: 0.6667
Epoch 5/20
3/3 [==============================] - 0s 75ms/step - loss: 2.1081 - accuracy: 0.4167 - val_loss: 1.9165 - val_accuracy: 0.3333
Epoch 6/20
3/3 [==============================] - 0s 100ms/step - loss: 1.2106 - accuracy: 0.5417 - val_loss: 0.8510 - val_accuracy: 0.3333
Epoch 7/20
3/3 [==============================] - 0s 82ms/step - loss: 1.4114 - accuracy: 0.5000 - val_loss: 1.0155 - val_accuracy: 0.6667
Epoch 8/20
3/3 [============

fine-tuning the pre-trained weights during the training process; update pre-trained model's weights by training it on specific dataset

In [45]:
# Freeze earlier layers and only fine-tune the later layers; model learn specific features relevant to dataset, preserving generic features learned during pre-training
# Freeze all layers first
for layer in base_model.layers:
    layer.trainable = False

In [46]:
# Gradual Unfreezing - progressively unfreeze earlier layers as the training progresses
# Unfreeze the last few layers
for layer in base_model.layers[-5:]:
    layer.trainable = True

In [47]:
# Compile the model with a lower learning rate
model.compile(optimizer=Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

In [48]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=8)

Epoch 1/20
3/3 [==============================] - 3s 202ms/step - loss: 5.2536 - accuracy: 0.5833 - val_loss: 0.7015 - val_accuracy: 0.3333
Epoch 2/20
3/3 [==============================] - 0s 86ms/step - loss: 0.7142 - accuracy: 0.6250 - val_loss: 0.6955 - val_accuracy: 0.3333
Epoch 3/20
3/3 [==============================] - 0s 87ms/step - loss: 0.8529 - accuracy: 0.5000 - val_loss: 0.7184 - val_accuracy: 0.3333
Epoch 4/20
3/3 [==============================] - 0s 86ms/step - loss: 0.7811 - accuracy: 0.3333 - val_loss: 0.6969 - val_accuracy: 0.3333
Epoch 5/20
3/3 [==============================] - 0s 86ms/step - loss: 0.6918 - accuracy: 0.5417 - val_loss: 0.7358 - val_accuracy: 0.3333
Epoch 6/20
3/3 [==============================] - 0s 86ms/step - loss: 0.7001 - accuracy: 0.5000 - val_loss: 0.7115 - val_accuracy: 0.3333
Epoch 7/20
3/3 [==============================] - 0s 89ms/step - loss: 0.6868 - accuracy: 0.5417 - val_loss: 0.7624 - val_accuracy: 0.3333
Epoch 8/20
3/3 [==========

fine-tuning the pre-trained weights during the training process; <br>
Unfreeze all the layers

In [49]:
for layer in base_model.layers:
    layer.trainable = True

In [50]:
# Compile the model with a lower learning rate
model.compile(optimizer=Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

In [51]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=8)

Epoch 1/20
3/3 [==============================] - 10s 289ms/step - loss: 0.6916 - accuracy: 0.5417 - val_loss: 0.6987 - val_accuracy: 0.3333
Epoch 2/20
3/3 [==============================] - 0s 159ms/step - loss: 0.6915 - accuracy: 0.5417 - val_loss: 0.6999 - val_accuracy: 0.3333
Epoch 3/20
3/3 [==============================] - 0s 173ms/step - loss: 0.6916 - accuracy: 0.5417 - val_loss: 0.7008 - val_accuracy: 0.3333
Epoch 4/20
3/3 [==============================] - 0s 159ms/step - loss: 0.6918 - accuracy: 0.5417 - val_loss: 0.7010 - val_accuracy: 0.3333
Epoch 5/20
3/3 [==============================] - 0s 157ms/step - loss: 0.6912 - accuracy: 0.5417 - val_loss: 0.7018 - val_accuracy: 0.3333
Epoch 6/20
3/3 [==============================] - 0s 159ms/step - loss: 0.6910 - accuracy: 0.5417 - val_loss: 0.7022 - val_accuracy: 0.3333
Epoch 7/20
3/3 [==============================] - 0s 159ms/step - loss: 0.6918 - accuracy: 0.5417 - val_loss: 0.7028 - val_accuracy: 0.3333
Epoch 8/20
3/3 [===

Early stopping and other metrics

In [52]:
from keras import metrics

f1_score = metrics.F1Score(name='f1_score')
precision = metrics.Precision(name='precision')
recall = metrics.Recall(name='recall')

In [53]:
# Compile the model
model.compile(optimizer=Adam(lr=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy', f1_score, precision, recall])

In [54]:
# Check data shapes
print('X_train shape:', X_train.shape)
print('X_val shape:', X_val.shape)
print('y_train shape:', y_train.shape)
print('y_val shape:', y_val.shape)

X_train shape: (24, 224, 224, 3)
X_val shape: (6, 224, 224, 3)
y_train shape: (24,)
y_val shape: (6,)


In [55]:
# Check data types
print('X_train dtype:', X_train.dtype)
print('X_val dtype:', X_val.dtype)
print('y_train dtype:', y_train.dtype)
print('y_val dtype:', y_val.dtype)

X_train dtype: float64
X_val dtype: float64
y_train dtype: int64
y_val dtype: int64


In [56]:
y_train = tf.convert_to_tensor(y_train, dtype = tf.float64)
y_val = tf.convert_to_tensor(y_val, dtype = tf.float64)

In [57]:
# Check data types
print('X_train dtype:', X_train.dtype)
print('X_val dtype:', X_val.dtype)
print('y_train dtype:', y_train.dtype)
print('y_val dtype:', y_val.dtype)

X_train dtype: float64
X_val dtype: float64
y_train dtype: <dtype: 'float64'>
y_val dtype: <dtype: 'float64'>


In [58]:
from keras.callbacks import EarlyStopping

In [59]:
early_stop = EarlyStopping(monitor='val_loss', patience=5)

model.fit(X_train, y_train,
          epochs=20,
          batch_size=8,
          validation_data=(X_val, y_val),
          callbacks=[early_stop])

Epoch 1/20
3/3 [==============================] - 6s 443ms/step - loss: 0.6920 - accuracy: 0.5417 - f1_score: 0.7027 - precision: 0.5417 - recall: 1.0000 - val_loss: 0.7090 - val_accuracy: 0.3333 - val_f1_score: 0.5000 - val_precision: 0.3333 - val_recall: 1.0000
Epoch 2/20
3/3 [==============================] - 0s 178ms/step - loss: 0.6887 - accuracy: 0.5417 - f1_score: 0.7027 - precision: 0.5417 - recall: 1.0000 - val_loss: 0.7093 - val_accuracy: 0.3333 - val_f1_score: 0.5000 - val_precision: 0.3333 - val_recall: 1.0000
Epoch 3/20
3/3 [==============================] - 0s 183ms/step - loss: 0.6913 - accuracy: 0.5417 - f1_score: 0.7027 - precision: 0.5417 - recall: 1.0000 - val_loss: 0.7096 - val_accuracy: 0.3333 - val_f1_score: 0.5000 - val_precision: 0.3333 - val_recall: 1.0000
Epoch 4/20
3/3 [==============================] - 1s 187ms/step - loss: 0.6927 - accuracy: 0.5417 - f1_score: 0.7027 - precision: 0.5417 - recall: 1.0000 - val_loss: 0.7096 - val_accuracy: 0.3333 - val_f1_sco

In [60]:
model.history.history

{'loss': [0.6919839382171631,
  0.6886863708496094,
  0.6913436055183411,
  0.6927111744880676,
  0.6900506019592285,
  0.6907815933227539],
 'accuracy': [0.5416666865348816,
  0.5416666865348816,
  0.5416666865348816,
  0.5416666865348816,
  0.5416666865348816,
  0.5416666865348816],
 'f1_score': [array([0.7027027], dtype=float32),
  array([0.7027027], dtype=float32),
  array([0.7027027], dtype=float32),
  array([0.7027027], dtype=float32),
  array([0.7027027], dtype=float32),
  array([0.7027027], dtype=float32)],
 'precision': [0.5416666865348816,
  0.5416666865348816,
  0.5416666865348816,
  0.5416666865348816,
  0.5416666865348816,
  0.5416666865348816],
 'recall': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 'val_loss': [0.7089884281158447,
  0.7093397974967957,
  0.7095756530761719,
  0.7096417546272278,
  0.7100308537483215,
  0.7103155255317688],
 'val_accuracy': [0.3333333432674408,
  0.3333333432674408,
  0.3333333432674408,
  0.3333333432674408,
  0.3333333432674408,
  0.333333343267440